In [10]:
import numpy as np
import pandas as pd

#gør at matricerne bliver printet nice, men laver permanent om på dine numpy indstillinger, så husk at ændre det tilbage
# pd.set_option('precision', 0)
np.set_printoptions(suppress=True)
np.set_printoptions(suppress=False)

# Little et al. Branch and Bounding
by big_mag et al.

In [11]:
# Skamløst stjålet og oversat fra Jens visual basic kode
def beregn_reduceret_matrix(OG_matrix, branch_list=[]):
    n = len(OG_matrix)
    
    # 1-indeksering og kopiere den originale matrice
    reduced_matrix = np.hstack((np.repeat(999, n + 1)[:, np.newaxis],
                                np.vstack((np.repeat(999, n), OG_matrix))))
    
    # det er det de kalder mig
    big_m = OG_matrix[1][1]
    
    prev_city = {}
    next_city = {}
    
    for branch in branch_list:
        from_city = branch[0]
        to_city = branch[1]
        
        if to_city > 0:  # hvis vi SKAL gå til kanten 
            # kanten (from_city, to_city) skal benyttes i løsningen
            prev_city[to_city] = from_city
            next_city[from_city] = to_city
            
            # eliminer alle andre kanter til to_city
            for i in range(1, n + 1):
                if i != from_city:
                    reduced_matrix[i][to_city] = big_m
                    
            # eliminer alle andre kanter til from_city
            for j in range(1, n + 1):
                if j != to_city:
                    reduced_matrix[from_city][j] = big_m
                    
        else:  # hvis vi IKKE MÅ gå til kanten
            reduced_matrix[from_city][-to_city] = big_m
          
    # forbyd kanten fra sidste til første by i hver krævet sti
    for start_city in range(1, n + 1):
        if prev_city.get(start_city, 0) == 0 and next_city.get(start_city, 0) > 0: 
            end_city = next_city.get(start_city)
            while next_city.get(end_city, 0) > 0:
                end_city = next_city.get(end_city)
            reduced_matrix[end_city][start_city] = big_m
           
    # gennemfør herefter reduktionen
    total_reduction = 0
    for i in range(1, n + 1):
        this_reduction = min(reduced_matrix[i])
        for j in range(1, n + 1):
            reduced_matrix[i][j] = reduced_matrix[i][j] - this_reduction
        total_reduction += this_reduction
        
    for j in range(1, n + 1):
        this_reduction = min(reduced_matrix[:, j])
        for i in range(1, n + 1):
            reduced_matrix[i][j] = reduced_matrix[i][j] - this_reduction
        total_reduction += this_reduction
    
    #fjerner 1-indeksering
    reduced_matrix = np.delete(np.delete(reduced_matrix, 0, 0), 0, 1)
    return reduced_matrix, total_reduction
    

In [12]:
def straf_matrix(reduced_matrix):
    # det hele forklarer lidt sig selv
    n = len(reduced_matrix)
    svar_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if reduced_matrix[i][j] == 0:
                min_row = min(reduced_matrix[i][np.arange(len(reduced_matrix)) != j])
                min_column = min(reduced_matrix[:, j][np.arange(len(reduced_matrix)) != i])
                svar_matrix[i][j] = min_row + min_column
                
    return svar_matrix

In [13]:
def print_table(reduceret_matrix):
    
    def color_red(val):  # vi maler byen rød
        color = 'red' if val > 500 else ''  # dog kun hvis den er større end 500
        return 'background-color: %s' % color
    
    n = len(reduceret_matrix[0])
    
    # 1-indeksere vores udskrift
    pd_df = pd.DataFrame(reduceret_matrix[0], 
                         columns=list(range(1,n + 1)),
                         index = list(range(1, n + 1)))
    
    print(f"Total reduktion: {reduceret_matrix[1]}")
    return pd_df.style.applymap(color_red) # vi applyer Birthe Kjær til matricen

In [14]:
def print_straf_table(straf_matrix):  # Total original kode
    
    matrix, (t, f) = straf_matrix
    print("Straf Matrix:")
    
    def style_specific_cell(x):  # vi gør punkt (t,f) grønt
        color = 'background-color: green'
        df1 = pd.DataFrame('', index=x.index, columns=x.columns)
        df1.iloc[t, f] = color
        return df1
    
    n = len(matrix)
    
    # 1-indeksere vores udskrift
    pd_df = pd.DataFrame(matrix, 
                         columns=list(range(1,n + 1)), 
                         index = list(range(1, n + 1)))
    
    return pd_df.style.apply(style_specific_cell, axis=None)  # smid den på matricen

In [15]:
def branch_and_bound(matrix, must_end_at=None):
    
    
    branch_costs = []  # liste til alle de branches vi støder på
    costs = []         # liste med costs, den sidste vil altid være den bedste
    reds = []          # liste med de reducerede matricer, den sidste vil altid være den bedste
    
    def solve(matrix, branch_list=[]):
        
        # danner den reducerede matrice og straf matricen
        red_matrix, cost = beregn_reduceret_matrix(matrix, branch_list)
        straf = straf_matrix(red_matrix)
        
        # vi gemmer resultaterne
        reds.append(red_matrix)
        costs.append(cost)
        
        # output nogle resultater
        print(f"{branch_list=}")
        display(print_table((red_matrix, cost)))
        
        
        if must_end_at: 
            # vi sørger for at vi ikke ser på den søjle vi skal slutte i
            tmp_straf = straf[:, [i for i in range(len(straf)) if i != (must_end_at - 1)]]
            
            kant = np.where(tmp_straf == tmp_straf[tmp_straf < 800].max())
            f = kant[0][0] + 1 # fra kant
            t = kant[1][0] + 1 # til kant
            
            # hvis vi ligger efter den søjle vi skal slutte i, bliver vi nødt til at ligge 1 til søjle indekset
            if t >= must_end_at:
                t += 1
                
        else:
            kant = np.where(straf == straf[straf < 800].max())
            f = kant[0][0] + 1  # fra kant
            t = kant[1][0] + 1  # til kant
            
        # vi gemmer den "venstre" branch, i tilfælde af at den optimale løsning ikke bare direkte til højre 
        # vi ved allerede nu at cost af denne gren bliver (straf[f - 1][t - 1] + cost)
        branch_costs.append(((straf[f - 1][t - 1] + cost), branch_list + [(f, -1 * t)]))
        
        # vi undgår at vise den allersidste straf matrice
        if straf[f - 1][t - 1] != 0:
            display(print_straf_table((straf, (f-1, t-1))))
            
        # Vi stopper hvis vi finder en fuld løsning og der ikke er findes en lavere cost
        if np.all(straf[straf < 800] == 0) and cost <= min(branch_costs, key=lambda x: x[0])[0]:
            return
        
        else:
            # hvis vores cost er den laveste fortsætter vi "blindt" til højre i træet
            if cost <= min(branch_costs, key=lambda x: x[0])[0]: 
                solve(matrix, branch_list + [(f, t)])  # Now do it again
            
            else:  # vores cost var shit, så vi går tilbage til tidligere branch muligheder og kigger dem igennem
                # vi skal lige huske at gemme det branch vi står, i tilfælde at de andre ruter også er trash
                branch_costs.append((cost, branch_list))
                
                # vi vælger den bedste branch tilgængelig
                h_branch = branch_costs.pop(branch_costs.index(min(branch_costs, key=lambda x: x[0])))
                solve(matrix, h_branch[1])
    
    solve(matrix)
    
    res = np.argwhere(reds[-1] == 0)  # vi finder alle indekserne på alle 0 i vores løsning
    
    return list(map(lambda x: (x[0]+1, x[1]+1), res)), costs[-1]

In [16]:
# HELTAL-1 2 som eksempel

m = 999

#                                      d
#                                      u
#                                      m
#                                      m
#                   A   B   C   D   E  y 
costs = np.array([[ m,  4,  9, 13,  7, 0],  # A
                  [14,  m, 12,  8,  5, 0],  # B
                  [10, 18,  m, 18, 17, 0],  # C
                  [11, 16,  8,  m, 15, 0],  # D
                  [ 3,  5,  2, 20,  m, 0],  # E
                  [ 0,  0,  0,  0,  0, m]]) # dummy
branch, cost = branch_and_bound(costs)
print(f"{branch=}, {cost=}")


branch_list=[]
Total reduktion: 0


,1,2,3,4,5,6
1,999,4,9,13,7,0
2,14,999,12,8,5,0
3,10,18,999,18,17,0
4,11,16,8,999,15,0
5,3,5,2,20,999,0
6,0,0,0,0,0,999


Straf Matrix:


,1,2,3,4,5,6
1,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
6,3.000000,4.000000,2.000000,8.000000,5.000000,0.000000


branch_list=[(3, 6)]
Total reduktion: 19


...

Straf Matrix:


,1,2,3,4,5,6
1,0.000000,1986.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,1985.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,1990.000000
4,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000
6,3.000000,0.000000,0.000000,18.000000,0.000000,0.000000


branch_list=[(3, 6), (1, 2), (2, 5), (5, 3)]
Total reduktion: 22


,1,2,3,4,5,6
1,995,0,995,995,995,995
2,994,994,994,994,0,994
3,999,999,999,999,999,0
4,0,988,988,988,988,988
5,997,997,0,997,997,997
6,999,999,999,0,999,999


branch=[(1, 2), (2, 5), (3, 6), (4, 1), (5, 3), (6, 4)], cost=22


In [17]:
# HELTAL-1 2 som eksempel

m = 999

#                                      d
#                                      u
#                                      m
#                                      m
#                   A   B   C   D   E  y 
costs = np.array([[ m,  4,  9, 13,  7],  # A
                  [14,  m, 12,  8,  5],  # B
                  [10, 18,  m, 18, 17],  # C
                  [11, 16,  8,  m, 15],  # D
                  [ 3,  5,  2, 20,  m]]) #E
                                        #dummy

branch, cost = branch_and_bound(costs)
print(f"{branch=}, {cost=}")

branch_list=[]
Total reduktion: 32


,1,2,3,4,5
1,995,0,5,6,3
2,9,994,7,0,0
3,0,8,989,5,7
4,3,8,0,988,7
5,1,3,0,15,997


Straf Matrix:


,1,2,3,4,5
1,0.000000,6.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,5.000000,3.000000
3,6.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,3.000000,0.000000,0.000000
5,0.000000,0.000000,1.000000,0.000000,0.000000


branch_list=[(1, 2)]
Total reduktion: 32


...

Straf Matrix:


,1,2,3,4,5
1,0.000000,0.000000,0.000000,0.000000,989.000000
2,0.000000,0.000000,0.000000,12.000000,0.000000
3,1977.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,1977.000000,0.000000,0.000000
5,0.000000,1001.000000,0.000000,0.000000,0.000000


branch_list=[(1, -2), (3, 1), (4, 3), (2, 4)]
Total reduktion: 38


,1,2,3,4,5
1,992,992,992,992,0
2,991,991,991,0,991
3,0,989,989,989,989
4,991,991,0,991,991
5,994,0,994,994,994


branch=[(1, 5), (2, 4), (3, 1), (4, 3), (5, 2)], cost=38


In [18]:
# HELTAL-1 2 som eksempel

m = 999


#                                      d
#                                      u
#                                      m
#                                      m
#                   A   B   C   D   E  y 
costs = np.array([[ m,  4,  9, 13,  7],  # A
                  [14,  m, 12,  8,  5],  # B
                  [10, 18,  m, 18, 17],  # C
                  [11, 16,  8,  m, 15],  # D
                  [ 3,  5,  2, 20,  m]]) # E #dummy

#                                      d
#                                      u
#                                      m
#                                      m
#                   A   B   C   D   E  y 
costs = np.array([[ m,  4,  9, 13,  7, 0],  # A
                  [14,  m, 12,  8,  5, 0],  # B
                  [10, 18,  m, 18, 17, 0],  # C
                  [11, 16,  8,  m, 15, 0],  # D
                  [ 3,  5,  2, 20,  m, 0], #E
                  [ 0,  0,  0,  0,  0, m]])  #dummy
                   
#                                      d
#                                      u
#                                      m
#                                      m
#                   A   B   C   D   E  y 
costs = np.array([[ m,  4,  9, 13,  7, 0],  # A
                  [14,  m, 12,  8,  5, 0],  # B
                  [10, 18,  m, 18, 17, 0],  # C
                  [11, 16,  8,  m, 15, 0],  # D
                  [ 3,  5,  2, 20,  m, 0], #E
                  [ m,  m,  m,  m,  m, m]])  #dummy

branch, cost = branch_and_bound(costs)
print(f"{branch=}, {cost=}")

branch_list=[]
Total reduktion: 999


,1,2,3,4,5,6
1,999,4,9,13,7,0
2,14,999,12,8,5,0
3,10,18,999,18,17,0
4,11,16,8,999,15,0
5,3,5,2,20,999,0
6,0,0,0,0,0,0


Straf Matrix:


,1,2,3,4,5,6
1,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
6,3.000000,4.000000,2.000000,8.000000,5.000000,0.000000


branch_list=[(3, 6)]
Total reduktion: 1018


...

Straf Matrix:


,1,2,3,4,5,6
1,0.000000,995.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,999.000000
4,0.000000,0.000000,3.000000,0.000000,0.000000,0.000000
5,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,991.000000,0.000000


branch_list=[(3, 6), (1, 2), (2, 5), (5, 3), (1, -1)]
Total reduktion: 1021


,1,2,3,4,5,6
1,995,0,995,995,995,995
2,994,994,994,994,0,994
3,999,999,999,999,999,0
4,0,988,988,988,988,988
5,997,997,0,997,997,997
6,0,0,0,0,0,0


branch=[(1, 2), (2, 5), (3, 6), (4, 1), (5, 3), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6)], cost=1021


In [19]:
# HELTAL-1 3 som eksempel

m = 999

#                                      d
#                                      u
#                                      m
#                                      m
#                   A   B   C   D   E  y 
costs = np.array([[ m,  4,  9, 13,  7, m],  # A
                  [14,  m, 12,  8,  5, m],  # B
                  [10, 18,  m, 14, 17, m],  # C
                  [11, 16,  8,  m, 15, m],  # D
                  [ 3,  5,  2, 20,  m, 0],  # E
                  [ 0,  m,  m,  m,  m, m]]) # dummy

#Sørg for at must_end og dummy variablen passer sammen!
#og husk at must_end er 1-indekseret
branch, cost = branch_and_bound(costs, must_end_at=5)
print(f"{branch=}, {cost=}")

branch_list=[]
Total reduktion: 30


,1,2,3,4,5,6
1,995,0,5,6,3,995
2,9,994,7,0,0,994
3,0,8,989,1,7,989
4,3,8,0,988,7,991
5,3,5,2,17,999,0
6,0,999,999,996,999,999


Straf Matrix:


,1,2,3,4,5,6
1,0.000000,8.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000
3,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,991.000000
6,996.000000,0.000000,0.000000,0.000000,0.000000,0.000000


branch_list=[(1, 2)]
Total reduktion: 30


...

Straf Matrix:


,1,2,3,4,5,6
1,0.000000,995.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,1.000000,4.000000,0.000000
3,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,0.000000,0.000000,997.000000,0.000000,0.000000,0.000000
6,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000


branch_list=[(1, 2), (4, 3), (2, 4), (1, -1)]
Total reduktion: 37


,1,2,3,4,5,6
1,995,0,995,995,995,995
2,991,991,991,0,991,991
3,982,982,982,982,0,982
4,991,991,0,991,991,991
5,3,999,999,999,999,0
6,0,999,999,999,999,999


branch=[(1, 2), (2, 4), (3, 5), (4, 3), (5, 6), (6, 1)], cost=37


In [20]:
# eksempel fra slides

#                                       d
#                                       u
#                                       m
#                                       m
#                   A   B   C   D   E   y                    
costs = np.array([[ m, 27, 43, 16, 30, 26],  # A
                  [ 7,  m, 16,  1, 30, 25],  # B
                  [20, 13,  m, 35,  5,  0],  # C
                  [21, 16, 25,  m, 18, 18],  # D
                  [12, 46, 27, 48,  m,  5],  # E
                  [23,  5,  5,  9,  5,  m]]) # dummy


#bare indsæt cost matricen og kryds fingre for at det ikke bare er en kæmpe prank:
branch, cost = branch_and_bound(costs)
print(f"{branch=}, {cost=}")

branch_list=[]
Total reduktion: 48


,1,2,3,4,5,6
1,978,11,27,0,14,10
2,1,998,15,0,29,24
3,15,13,999,35,5,0
4,0,0,9,983,2,2
5,2,41,22,43,994,0
6,13,0,0,4,0,994


Straf Matrix:


,1,2,3,4,5,6
1,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
4,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
6,0.000000,0.000000,9.000000,0.000000,2.000000,0.000000


branch_list=[(1, 4)]
Total reduktion: 49


...

Straf Matrix:


,1,2,3,4,5,6
1,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,1968.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000
4,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
6,0.000000,0.000000,1964.000000,0.000000,0.000000,0.000000


branch_list=[(1, 4), (2, 1), (5, 6), (3, 5), (1, -1)]
Total reduktion: 63


,1,2,3,4,5,6
1,983,983,983,0,983,983
2,0,992,992,992,992,992
3,994,994,994,994,0,994
4,974,974,0,974,974,974
5,994,994,994,994,994,0
6,994,0,994,994,994,994


branch=[(1, 4), (2, 1), (3, 5), (4, 3), (5, 6), (6, 2)], cost=63


In [48]:
import pandas as pd
import numpy as np
import matplotlib as mpl

In [49]:
def print_table(reduceret_matrix):
    
    def color_red(val):  # vi maler byen rød
        color = 'red' if val > 500 else ''  # dog kun hvis den er større end 500
        return 'background-color: %s' % color
    
    n = len(reduceret_matrix)
    
    # 1-indeksere vores udskrift
    pd_df = pd.DataFrame(reduceret_matrix)
    
    print(pd_df)
    return pd_df.style.applymap(color_red) # vi applyer Birthe Kjær til matricen

In [106]:
def print_table(mat):
    pd_df = pd.DataFrame(mat)
    s = pd_df.style.format()
    
    s.set_table_styles([  # create internal CSS classes
    {'selector': '.re', 'props': 'background-color: #FF0000;'},
    {'selector': '.bl', 'props': 'background-color: #0000FF;'},
    {'selector': '.or', 'props': 'background-color: #FFA500;'},
    {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
    ], overwrite=False)
    
    cell_color = pd.DataFrame([['re ', 'bl ', 're '],
                               ['bl ', 'bl ', 'bl '],
                               ['re ', 'or ', 're ']],
                          index=pd_df.index,
                          columns=pd_df.columns[:10])
    s.set_td_classes(cell_color)
    
    return s

In [107]:
v = np.array([[1, 2, 3], [999, 999, 999], [3, 3, 3]])

print_table(v)

,0,1,2
0,1,2,3
1,999,999,999
2,3,3,3


In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl

df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
df.style

In [3]:
df.style.format(precision=0, na_rep='MISSING', thousands=" ",
                formatter={('Decision Tree', 'Tumour'): "{:.2f}",
                           ('Regression', 'Non-Tumour'): lambda x: "$ {:,.1f}".format(x*-1e6)
                          })

In [4]:
s = df.style.format('{:.0f}').hide([('Random', 'Tumour'), ('Random', 'Non-Tumour')], axis="columns")
s

In [10]:
import numpy as np

def print_table(mat, sup, tar, i, j, s, t):
    mat = np.concatenate((mat, [tar]), axis = 0)
    pd_df = pd.DataFrame(mat)
    sup = np.concatenate((sup, [0]))
    pd_df["supply"] = sup
    q = pd_df.style.format()
    
    q.set_table_styles([  # create internal CSS classes
    {'selector': '.re', 'props': 'background-color: #FF0000;'},
    {'selector': '.bl', 'props': 'background-color: #0000FF;'},
    {'selector': '.or', 'props': 'background-color: #FFA500;'},
    {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
    ], overwrite=False)
    
    
    le = len(mat[0])
    c_mat = np.array([['bb ' if i != le else 'or ' for i in range(le + 1)] for _ in range(len(mat) - 1)])
    ex = [['or ' for i in range(le)] + ['bb ']]
    
    c_mat[i, j] = 're '

    c_mat = np.concatenate((c_mat, ex), axis = 0)
    
    for i in range(len(s)):
        if not(s[i]):
            c_mat[i] = 'bl '
    
    for i in range(len(t)):
        if not(t[i]):
            c_mat[..., i] = 'bl '
    
    print(c_mat)
    
    print(pd_df.columns[:100])
    
    #cell_color = pd.DataFrame(np.array([['re ', 're ', 're ', 're', 'or '],
    #                           ['re ', 'bl ', 'bl ', 're', 'or '],
    #                           ['re ', 're ', 're ', 're', 'or '],
    #                           ['or ', 'or ', 'or ', 'or ', 'bb ']]),
    #                      index=pd_df.index,
    #                      columns=pd_df.columns[:10])
    
    cell_color = pd.DataFrame(c_mat,
                          index=pd_df.index,
                          columns=pd_df.columns[:10])
    q.set_td_classes(cell_color)

    return q


supply = np.array([6, 3, 5])
demand = np.array([2, 3, 2, 7])
t = np.array([0, 1, 1, 1, 1])
s = np.array([1, 1, 1])

supply = np.array([5, 5, 6])
demand = np.array([2, 2, 4, 4, 4])
costs = np.array([[10, 1, 1, 1, 1],
         [1, 1, 1, 1, 1],
         [1, 1, 1, 1, 50]])

print_table(costs, supply, demand, 1, 0, s, t)


[['bl ' 'bb ' 'bb ' 'bb ' 'bb ' 'or ']
 ['bl ' 'bb ' 'bb ' 'bb ' 'bb ' 'or ']
 ['bl ' 'bb ' 'bb ' 'bb ' 'bb ' 'or ']
 ['bl ' 'or ' 'or ' 'or ' 'or ' 'bb ']]
Index([0, 1, 2, 3, 4, 'supply'], dtype='object')


,0,1,2,3,4,supply
0,10,1,1,1,1,5
1,1,1,1,1,1,5
2,1,1,1,1,50,6
3,2,2,4,4,4,0


In [278]:
def vogel(S, T, costs):

    def nan_gen(it, obj):
        ls = np.array([])
        j = 0
        for i in range(len(it)):
            if it[i]:
                ls = np.append(ls, obj[j])
                j += 1
            else:
                ls = np.append(ls, np.nan)
        return ls

    s = [True for _ in range(len(S))]
    t = [True for _ in range(len(T))]

    sup = np.array(S)
    tar = np.array(T)
    cost = np.array(costs)
    out = np.array([[0 for _ in range(sum(t))] for __ in range(sum(s))])


    while((sum(s) >= 2) and (sum(t) >= 2)):
        cst = cost[s][..., t]
        sorted = np.sort(cst, axis = 1)
        diff_r = sorted[..., 1] - sorted[..., 0]
        sorted = np.sort(cst, axis = 0)
        diff_c = sorted[1] - sorted[0]

        d_r = nan_gen(s, diff_r)
        d_c = nan_gen(t, diff_c)

        diff = [d_r, d_c]

        m = map(np.nanmax, diff)

        if next(m) >= next(m): # if a diff on supply is larger than on demand
            idx_r = np.nanargmax(d_r)
            idx_c = np.nanargmin(nan_gen(t, cost[idx_r, t]))
            send = min([sup[idx_r], tar[idx_c]])
            display(print_table(cost, sup, tar, idx_r, idx_c, s, t))
            out[idx_r, idx_c] = send

            sup[idx_r] = sup[idx_r] - send
            tar[idx_c] = tar[idx_c] - send
        else:
            idx_c = np.nanargmax(d_c)
            idx_r = np.nanargmin(nan_gen(s, cost[s, idx_c]))
            send = min([sup[idx_r], tar[idx_c]])
            display(print_table(cost, sup, tar, idx_r, idx_c, s, t))
            out[idx_r, idx_c] = send

            sup[idx_r] = sup[idx_r] - send
            tar[idx_c] = tar[idx_c] - send

        
              
        for i in range(len(sup)):
            if sup[i] == 0:
                s[i] = False

        for i in range(len(tar)):
            if tar[i] == 0:
                t[i] = False


    flag = False
    if sum(sup != 0) != 1:
        flag = True
        sup, tar = tar, sup
        cost = cost.T
        out = out.T
        t, s = s, t

    idx_r = np.where(sup != 0)

    for i in range(sum(tar != 0)):
        idx_c = np.nanargmin(nan_gen(t, cost[idx_r, t][0]))
        send = min([sup[idx_r], tar[idx_c]])
        out[idx_r, idx_c] = send

        sup[idx_r] = sup[idx_r] - send
        tar[idx_c] = tar[idx_c] - send

        for i in range(len(tar)):
            if tar[i] == 0:
                t[i] = False

    if flag:
        out = out.T
        cost = cost.T

    print("Vogels approximation:")
    print(out)

    print()
    print("Cost of this is:")
    print(np.sum(np.array(out) * np.array(cost)))

    return out


In [280]:
supply = [6, 3, 5]
demand = [2, 3, 2, 7]

costs = [[7, 11, 3, 8],
         [2, 10, 1, 2],
         [14, 18, 8, 5]]

d = vogel(supply, demand, costs)

,0,1,2,3,supply
0,7,11,3,8,6
1,2,10,1,2,3
2,14,18,8,5,5
3,2,3,2,7,0


,0,1,2,3,supply
0,7,11,3,8,6
1,2,10,1,2,1
2,14,18,8,5,5
3,0,3,2,7,0


,0,1,2,3,supply
0,7,11,3,8,4
1,2,10,1,2,1
2,14,18,8,5,5
3,0,3,0,7,0


,0,1,2,3,supply
0,7,11,3,8,4
1,2,10,1,2,1
2,14,18,8,5,0
3,0,3,0,2,0


Vogels approximation:
[[0 3 2 1]
 [2 0 0 1]
 [0 0 0 5]]

Cost of this is:
78
